# Meeting Notes to Google Doc Converter

This notebook converts markdown meeting notes into a well-formatted Google Document.

## Setup Instructions

1. Run all cells in order
2. Authenticate with your Google account when prompted
3. The script will create a new Google Doc and provide you with the URL


## Step 1: Install Required Dependencies


In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


## Step 2: Import Libraries


In [ ]:
import re
from google.colab import auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import google.auth


## Step 3: Define Meeting Notes

The markdown meeting notes are stored as a string variable.


In [ ]:
MEETING_NOTES = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


## Step 4: Markdown Parser Class

This class parses markdown content and converts it into structured data.


In [ ]:
class MarkdownParser:
    """
    Parses markdown text and extracts structured elements.
    Identifies headings, bullet points, checkboxes, and mentions.
    """
    
    def __init__(self, markdown_text):
        """Initialize parser with markdown text."""
        self.markdown_text = markdown_text
        self.elements = []
    
    def parse(self):
        """Parse markdown text into structured elements."""
        lines = self.markdown_text.split('\n')
        
        for line in lines:
            if not line.strip():
                # Skip empty lines
                continue
            
            # Check for horizontal rule (footer separator)
            if line.strip() == '---':
                self.elements.append({
                    'type': 'separator',
                    'text': ''
                })
                continue
            
            # Check for headings (H1)
            if line.startswith('# '):
                self.elements.append({
                    'type': 'heading1',
                    'text': line[2:].strip()
                })
            # Check for headings (H2)
            elif line.startswith('## '):
                self.elements.append({
                    'type': 'heading2',
                    'text': line[3:].strip()
                })
            # Check for headings (H3)
            elif line.startswith('### '):
                self.elements.append({
                    'type': 'heading3',
                    'text': line[4:].strip()
                })
            # Check for checkbox items
            elif re.match(r'^- \[ \]', line):
                indent_level = self._get_indent_level(line)
                text = re.sub(r'^- \[ \]\s*', '', line.lstrip())
                self.elements.append({
                    'type': 'checkbox',
                    'text': text,
                    'indent': indent_level,
                    'mentions': self._extract_mentions(text)
                })
            # Check for bullet points (- or *)
            elif re.match(r'^\s*[\-\*]\s+', line):
                indent_level = self._get_indent_level(line)
                text = re.sub(r'^\s*[\-\*]\s+', '', line)
                self.elements.append({
                    'type': 'bullet',
                    'text': text,
                    'indent': indent_level
                })
            # Regular text (footer information)
            else:
                self.elements.append({
                    'type': 'text',
                    'text': line.strip()
                })
        
        return self.elements
    
    def _get_indent_level(self, line):
        """Calculate indentation level based on leading spaces."""
        # Count leading spaces
        spaces = len(line) - len(line.lstrip())
        # Every 2 spaces = 1 indent level
        return spaces // 2
    
    def _extract_mentions(self, text):
        """Extract @mentions from text."""
        return re.findall(r'@\w+', text)


## Step 5: Google Docs Formatter Class

This class handles the conversion from parsed markdown to Google Docs formatting.


In [ ]:
class GoogleDocsFormatter:
    """
    Converts parsed markdown elements into Google Docs API requests.
    Handles formatting, styling, and special elements like checkboxes.
    """
    
    def __init__(self, service):
        """Initialize formatter with Google Docs API service."""
        self.service = service
        self.requests = []
        self.current_index = 1  # Start at index 1 (after document start)
    
    def create_document(self, title):
        """Create a new Google Doc and return document ID."""
        try:
            doc = self.service.documents().create(body={'title': title}).execute()
            return doc.get('documentId')
        except HttpError as error:
            print(f'An error occurred creating document: {error}')
            raise
    
    def format_elements(self, elements):
        """Convert parsed elements into Google Docs API requests."""
        self.requests = []
        self.current_index = 1
        
        for element in elements:
            element_type = element['type']
            
            if element_type == 'heading1':
                self._add_heading(element['text'], 'HEADING_1')
            elif element_type == 'heading2':
                self._add_heading(element['text'], 'HEADING_2')
            elif element_type == 'heading3':
                self._add_heading(element['text'], 'HEADING_3')
            elif element_type == 'bullet':
                self._add_bullet(element['text'], element['indent'])
            elif element_type == 'checkbox':
                self._add_checkbox(element['text'], element['indent'], element['mentions'])
            elif element_type == 'separator':
                # Add a blank line for separator
                self._add_text('\n')
            elif element_type == 'text':
                self._add_footer_text(element['text'])
        
        return self.requests
    
    def _add_heading(self, text, style):
        """Add a heading with specified style."""
        # Insert text
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text + '\n'
            }
        })
        
        # Apply heading style
        end_index = self.current_index + len(text)
        self.requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': self.current_index,
                    'endIndex': end_index + 1
                },
                'paragraphStyle': {
                    'namedStyleType': style
                },
                'fields': 'namedStyleType'
            }
        })
        
        self.current_index += len(text) + 1
    
    def _add_bullet(self, text, indent_level):
        """Add a bullet point with proper indentation."""
        # Insert text
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text + '\n'
            }
        })
        
        # Apply bullet formatting
        end_index = self.current_index + len(text)
        self.requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': self.current_index,
                    'endIndex': end_index + 1
                },
                'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
            }
        })
        
        # Apply indentation if needed
        if indent_level > 0:
            self.requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': self.current_index,
                        'endIndex': end_index + 1
                    },
                    'paragraphStyle': {
                        'indentStart': {'magnitude': 36 * indent_level, 'unit': 'PT'},
                        'indentFirstLine': {'magnitude': 18, 'unit': 'PT'}
                    },
                    'fields': 'indentStart,indentFirstLine'
                }
            })
        
        self.current_index += len(text) + 1
    
    def _add_checkbox(self, text, indent_level, mentions):
        """Add a checkbox item with mentions styled."""
        # Insert text
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text + '\n'
            }
        })
        
        # Apply checkbox (bullet with checkbox glyph)
        end_index = self.current_index + len(text)
        self.requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': self.current_index,
                    'endIndex': end_index + 1
                },
                'bulletPreset': 'BULLET_CHECKBOX'
            }
        })
        
        # Style mentions (@name) in bold and blue
        for mention in mentions:
            mention_start = text.find(mention)
            if mention_start != -1:
                mention_abs_start = self.current_index + mention_start
                mention_abs_end = mention_abs_start + len(mention)
                
                # Make mention bold
                self.requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': mention_abs_start,
                            'endIndex': mention_abs_end
                        },
                        'textStyle': {
                            'bold': True,
                            'foregroundColor': {
                                'color': {
                                    'rgbColor': {
                                        'red': 0.0,
                                        'green': 0.4,
                                        'blue': 0.8
                                    }
                                }
                            }
                        },
                        'fields': 'bold,foregroundColor'
                    }
                })
        
        self.current_index += len(text) + 1
    
    def _add_text(self, text):
        """Add plain text."""
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text
            }
        })
        self.current_index += len(text)
    
    def _add_footer_text(self, text):
        """Add footer text with italic styling."""
        # Insert text
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text + '\n'
            }
        })
        
        # Apply italic style
        end_index = self.current_index + len(text)
        self.requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': self.current_index,
                    'endIndex': end_index
                },
                'textStyle': {
                    'italic': True,
                    'foregroundColor': {
                        'color': {
                            'rgbColor': {
                                'red': 0.4,
                                'green': 0.4,
                                'blue': 0.4
                            }
                        }
                    }
                },
                'fields': 'italic,foregroundColor'
            }
        })
        
        self.current_index += len(text) + 1
    
    def apply_formatting(self, document_id, requests):
        """Apply all formatting requests to the document."""
        try:
            result = self.service.documents().batchUpdate(
                documentId=document_id,
                body={'requests': requests}
            ).execute()
            return result
        except HttpError as error:
            print(f'An error occurred applying formatting: {error}')
            raise


## Step 6: Main Conversion Function


In [ ]:
def convert_markdown_to_google_doc(markdown_text, doc_title="Meeting Notes"):
    """
    Main function to convert markdown text to a Google Doc.
    
    Args:
        markdown_text: String containing markdown content
        doc_title: Title for the new Google Doc
    
    Returns:
        Document ID and URL of the created Google Doc
    """
    try:
        # Step 1: Authenticate
        print("Authenticating with Google...")
        auth.authenticate_user()
        creds, _ = google.auth.default()
        
        # Step 2: Build the Docs service
        print("Building Google Docs service...")
        service = build('docs', 'v1', credentials=creds)
        
        # Step 3: Parse markdown
        print("Parsing markdown content...")
        parser = MarkdownParser(markdown_text)
        elements = parser.parse()
        print(f"Parsed {len(elements)} elements")
        
        # Step 4: Create Google Doc
        print(f"Creating Google Doc: '{doc_title}'...")
        formatter = GoogleDocsFormatter(service)
        document_id = formatter.create_document(doc_title)
        print(f"Document created with ID: {document_id}")
        
        # Step 5: Format content
        print("Formatting content...")
        requests = formatter.format_elements(elements)
        print(f"Generated {len(requests)} formatting requests")
        
        # Step 6: Apply formatting
        print("Applying formatting to document...")
        formatter.apply_formatting(document_id, requests)
        
        # Step 7: Return document info
        doc_url = f"https://docs.google.com/document/d/{document_id}/edit"
        print("\n" + "="*60)
        print("SUCCESS! Your Google Doc has been created.")
        print("="*60)
        print(f"Document ID: {document_id}")
        print(f"Document URL: {doc_url}")
        print("="*60)
        
        return document_id, doc_url
        
    except HttpError as error:
        print(f"HTTP Error occurred: {error}")
        print("Please check your authentication and API permissions.")
        raise
    except Exception as error:
        print(f"An unexpected error occurred: {error}")
        raise


## Step 7: Run the Conversion

Execute this cell to convert the meeting notes to a Google Doc.


In [ ]:
# Run the conversion
document_id, document_url = convert_markdown_to_google_doc(
    markdown_text=MEETING_NOTES,
    doc_title="Product Team Sync - May 15, 2023"
)

# Display clickable link
from IPython.display import display, HTML
display(HTML(f'<a href="{document_url}" target="_blank">Click here to open your Google Doc</a>'))
